## SCAD, pytorch, variable selection

In [2]:
!pip install torch

In [5]:
import torch

In [7]:
device = torch.device('cpu')
dtype = torch.float64

In [9]:
def scad_penalty(beta_hat, lambda_val, a_val):
    abs_beta_hat = torch.abs(beta_hat)
    is_linear = (abs_beta_hat <= lambda_val) # defining the flat outer bounds
    is_quadratic = torch.logical_and(lambda_val < abs_beta_hat, abs_beta_hat <= a_val * lambda_val) # if lambda < absolute value of beta <= a * lambda
    is_constant = (a_val * lambda_val) < abs_beta_hat # if absolute value of beta is greater than a * lambda
    
    linear_part = lambda_val * abs_beta_hat * is_linear
    quadratic_part = (2* a_val * lambda_val * abs_beta_hat - beta_hat**2 - lambda_val**2) / (2*(a_val - 1)) * is_quadratic
    constant_part = (lambda_val**2*(a_val+1)) / 2 * is_constant

    return linear_part + quadratic_part + constant_part

def scad_derivative(beta_hat, lambda_val, a_val):
    return lambda_val*((beta_hat <= lambda_val) + (a_val*lambda_val-beta_hat)*((a_val*lambda_val-beta_hat) > 0)/((a_val-1)*lambda_val)*(beta_hat>lambda_val))